## L3 - Gold Stage: Business-Ready Data Marts

**Tujuan Notebook:**
 
Notebook ini adalah tahap akhir dalam pipeline ETL, di mana data dari layer **Silver** yang sudah bersih dan terstruktur diubah menjadi **data mart** di layer **Gold**. Tabel Gold bersifat teragregasi, denormalisasi, dan dioptimalkan untuk kebutuhan analitik dan visualisasi dashboard.

**Proses yang Dilakukan:**
1.  **Load Data Silver**: Memuat semua tabel yang relevan dari layer Silver.
2.  **Buat Base View Terdenormalisasi**: Menggabungkan tabel-tabel utama (claim, policyholder, incharge, service) untuk membuat satu DataFrame dasar yang kaya akan fitur. Ini mencegah join berulang dan menjadi fondasi untuk semua agregasi.
3.  **Feature Engineering**: Membuat kolom-kolom baru yang berguna untuk analisis, seperti kategori risiko, kategori durasi, dan periode waktu (tahun-bulan).
4.  **Agregasi Data Mart**: Membuat beberapa tabel Gold yang terfokus pada kasus penggunaan spesifik:
    - `gold_claim_details`: Tabel transaksi terdenormalisasi.
    - `gold_policyholder_analytics`: Menganalisis perilaku dan risiko pemegang polis.
    - `gold_service_performance_monthly`: Menganalisis kinerja provider layanan.
    - `gold_overall_kpi`: Menyediakan Key Performance Indicators (KPI) tingkat tinggi.
    - `gold_disease_analytics` : Agregasi klaim per penyakit.
5.  **Simpan ke Gold Layer**: Menyimpan data mart sebagai tabel Delta yang siap digunakan oleh Power BI atau tool BI lainnya.


In [17]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Impor semua fungsi yang akan digunakan
from pyspark.sql.functions import (
    col, count, sum, avg, min, max, round, stddev, year, month, date_format, 
    concat_ws, when, countDistinct, current_timestamp
)

# Konfigurasi Path
silver_base_path = "Tables/Silver"
gold_base_path = "Tables/Gold"


StatementMeta(, d7712a4c-5fe9-44b1-aca7-459404fc68d8, 19, Finished, Available, Finished)

In [18]:
# Load Semua Tabel Silver yang Relevan
print("Loading tables from Silver layer...")
try:
    claim_df = spark.read.format("delta").load(f"{silver_base_path}/claim")
    policyholder_df = spark.read.format("delta").load(f"{silver_base_path}/policyholder")
    incharge_df = spark.read.format("delta").load(f"{silver_base_path}/incharge")
    service_df = spark.read.format("delta").load(f"{silver_base_path}/service")
    has_disease_df = spark.read.format("delta").load(f"{silver_base_path}/has_disease")
    disease_df = spark.read.format("delta").load(f"{silver_base_path}/disease")
    print("✅ All Silver tables loaded successfully.")
except Exception as e:
    print(f"❌ Error loading Silver tables: {e}")
    # Hentikan eksekusi jika tabel dasar tidak ada
    # dbutils.notebook.exit("Failed to load Silver tables")


StatementMeta(, d7712a4c-5fe9-44b1-aca7-459404fc68d8, 20, Finished, Available, Finished)

Loading tables from Silver layer...
✅ All Silver tables loaded successfully.


In [19]:
# Buat Base View Terdenormalisasi & Feature Engineering
print("Creating denormalized base view with feature engineering...")

# Gabungkan policyholder dan incharge untuk full name
policyholder_df = policyholder_df.withColumn("policyholder_full_name", concat_ws(" ", col("first_name"), col("last_name")))
incharge_df = incharge_df.withColumn("incharge_full_name", concat_ws(" ", col("first_name"), col("last_name")))

# Menggabungkan semua tabel menggunakan LEFT JOIN dan melakukan feature engineering
base_df = claim_df.alias("c") \
    .join(policyholder_df.alias("ph"), col("c.policyholder_id") == col("ph.policyholder_id"), "left") \
    .join(incharge_df.alias("i"), col("c.person_incharge_id") == col("i.incharge_id"), "left") \
    .join(service_df.alias("s"), col("i.service_id") == col("s.service_id"), "left") \
    .join(has_disease_df.alias("hd"), col("c.insured_id") == col("hd.patient_id"), "left") \
    .join(disease_df.alias("d"), col("hd.disease_id") == col("d.disease_id"), "left") \
    .select(
        col("c.claim_id"), col("c.claim_date"), col("c.charge"), col("c.duration"), col("c.insured_id"),
        col("ph.policyholder_id"), col("ph.policyholder_full_name"), col("ph.risk_score").alias("policyholder_risk_score"), col("ph.high_risk"),
        col("s.service_id"), col("s.service_name"),
        col("i.incharge_id"), col("i.incharge_full_name"),
        col("d.disease_id"), col("d.concept_name").alias("disease_name")
    ) \
    .withColumn( # Feature Engineering & Kolom Validasi
        "is_customer_valid", when(col("policyholder_id").isNotNull(), True).otherwise(False)
    ).withColumn(
        "is_disease_valid", when(col("disease_id").isNotNull(), True).otherwise(False)
    ).withColumn(
        "claim_year_month", date_format(col("claim_date"), "yyyy-MM")
    ).withColumn(
        "claim_year", year(col("claim_date"))
    ).withColumn(
        "risk_category", when(col("high_risk") == 1, "High Risk").otherwise("Low Risk")
    ).withColumn(
        "duration_category", 
        when(col("duration") <= 10, "Fast")
        .when(col("duration") <= 20, "Medium")
        .otherwise("Slow")
    )

# Drop duplicates berdasarkan claim_id untuk memastikan setiap klaim hanya dihitung sekali
base_df = base_df.dropDuplicates(["claim_id"])

base_df.cache()
print("✅ Base view created and cached.")
base_df.printSchema()


StatementMeta(, d7712a4c-5fe9-44b1-aca7-459404fc68d8, 21, Finished, Available, Finished)

Creating denormalized base view with feature engineering...
✅ Base view created and cached.
root
 |-- claim_id: string (nullable = true)
 |-- claim_date: date (nullable = true)
 |-- charge: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- insured_id: string (nullable = true)
 |-- policyholder_id: string (nullable = true)
 |-- policyholder_full_name: string (nullable = true)
 |-- policyholder_risk_score: integer (nullable = true)
 |-- high_risk: integer (nullable = true)
 |-- service_id: string (nullable = true)
 |-- service_name: string (nullable = true)
 |-- incharge_id: string (nullable = true)
 |-- incharge_full_name: string (nullable = true)
 |-- disease_id: string (nullable = true)
 |-- disease_name: string (nullable = true)
 |-- is_customer_valid: boolean (nullable = false)
 |-- is_disease_valid: boolean (nullable = false)
 |-- claim_year_month: string (nullable = true)
 |-- claim_year: integer (nullable = true)
 |-- risk_category: string (nullable = false)


In [20]:
# Buat Data Mart: gold_claim_details
print("\nCreating Data Mart: gold_claim_details...")

# Tabel ini adalah base_df itu sendiri, tanpa agregasi.
gold_claim_details = base_df.withColumn("update_date", current_timestamp())

gold_claim_details.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{gold_base_path}/claim_details")
print("✅ Successfully created 'gold_claim_details'.")
display(gold_claim_details.limit(5))


StatementMeta(, d7712a4c-5fe9-44b1-aca7-459404fc68d8, 22, Finished, Available, Finished)


Creating Data Mart: gold_claim_details...
✅ Successfully created 'gold_claim_details'.


SynapseWidget(Synapse.DataFrame, ddc70929-8499-4977-baf3-79c4065582ee)

In [21]:
# Buat Data Mart: gold_policyholder_analytics
print("\nCreating Data Mart: gold_policyholder_analytics...")

gold_policyholder_analytics = base_df.groupBy(
    "policyholder_id",
    "policyholder_full_name",
    "risk_category",
    "claim_year",
    "claim_year_month"
).agg(
    count("claim_id").alias("total_claims"),
    round(sum("charge"), 2).alias("total_charge_amount"),
    round(avg("duration"), 2).alias("average_claim_duration_days"),
    # Ambil nilai risk score pertama karena sama untuk policyholder yang sama
    max("policyholder_risk_score").alias("risk_score")
).withColumn("update_date", current_timestamp())

# Simpan ke Gold Layer
gold_policyholder_analytics.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{gold_base_path}/policyholder_analytics")
print("✅ Successfully created 'gold_policyholder_analytics'.")
display(gold_policyholder_analytics.limit(5))


StatementMeta(, d7712a4c-5fe9-44b1-aca7-459404fc68d8, 23, Finished, Available, Finished)


Creating Data Mart: gold_policyholder_analytics...
✅ Successfully created 'gold_policyholder_analytics'.


SynapseWidget(Synapse.DataFrame, 1d025be3-68a5-4e82-a6ce-40e999a52fc3)

In [22]:
# Buat Data Mart: gold_service_performance_monthly
print("\nCreating Data Mart: gold_service_performance_monthly...")

gold_service_performance = base_df.groupBy(
    "service_id",
    "service_name",
    "claim_year_month"
).agg(
    count("claim_id").alias("total_claims"),
    round(sum("charge"), 2).alias("total_charge_amount"),
    round(avg("duration"), 2).alias("average_claim_duration_days")
)

# Menambahkan kategori kecepatan proses (dari Bima)
gold_service_performance_monthly = gold_service_performance.withColumn(
    "processing_speed_category",
    when(col("average_claim_duration_days") <= 10, "Fast")
    .when(col("average_claim_duration_days") <= 20, "Medium")
    .otherwise("Slow")
).withColumn("update_date", current_timestamp())

# Simpan ke Gold Layer
gold_service_performance_monthly.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{gold_base_path}/service_performance_monthly")
print("✅ Successfully created 'gold_service_performance_monthly'.")
display(gold_service_performance_monthly.limit(5))


StatementMeta(, d7712a4c-5fe9-44b1-aca7-459404fc68d8, 24, Finished, Available, Finished)


Creating Data Mart: gold_service_performance_monthly...
✅ Successfully created 'gold_service_performance_monthly'.


SynapseWidget(Synapse.DataFrame, c1e6d8ce-f26c-4889-a0bb-4384648df079)

In [23]:
# Buat Data Mart: gold_overall_kpi
print("\nCreating Data Mart: gold_overall_kpi...")

gold_overall_kpi = base_df.groupBy("claim_year_month").agg(
    count("claim_id").alias("total_claims"),
    round(sum("charge"), 2).alias("total_charge_amount"),
    countDistinct("policyholder_id").alias("total_unique_customers"),
    # Asumsikan 'disease' akan di-join jika perlu
    # countDistinct("disease_id").alias("total_unique_diseases")
).withColumn("update_date", current_timestamp()).orderBy("claim_year_month")


# Simpan ke Gold Layer
gold_overall_kpi.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{gold_base_path}/overall_kpi")
print("✅ Successfully created 'gold_overall_kpi'.")
display(gold_overall_kpi.limit(5))


StatementMeta(, d7712a4c-5fe9-44b1-aca7-459404fc68d8, 25, Finished, Available, Finished)


Creating Data Mart: gold_overall_kpi...
✅ Successfully created 'gold_overall_kpi'.


SynapseWidget(Synapse.DataFrame, f861fd50-f78b-4ba8-aae0-56d63933c5b7)

In [27]:
# Buat Data Mart: gold_disease_analytics
print("\nCreating Data Mart: gold_disease_analytics...")
# Re-join khusus untuk analitik penyakit untuk memastikan semua data terhitung.
# Kita gunakan INNER JOIN karena hanya tertarik pada klaim yang memiliki penyakit valid.
disease_claims = claim_df.alias("c") \
    .join(has_disease_df.alias("hd"), col("c.insured_id") == col("hd.patient_id"), "inner") \
    .join(disease_df.alias("d"), col("hd.disease_id") == col("d.disease_id"), "inner")

# Hitung KPI total klaim dengan penyakit valid (sekarang akan akurat)
valid_disease_claim_count = disease_claims.count()
print(f"📊 KPI - Total Claims with a Valid Disease: {valid_disease_claim_count}")

# Agregasi untuk analisis Top 10 Penyakit
gold_disease_analytics = disease_claims \
    .groupBy("d.disease_id", "d.concept_name") \
    .agg(countDistinct("c.claim_id").alias("total_claims")) \
    .withColumn("update_date", current_timestamp()) \
    .orderBy(col("total_claims").desc())

gold_disease_analytics.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{gold_base_path}/disease_analytics")
print("✅ Successfully created 'gold_disease_analytics'.")
display(gold_disease_analytics.limit(5))

# Membersihkan cache
base_df.unpersist()


StatementMeta(, d7712a4c-5fe9-44b1-aca7-459404fc68d8, 29, Finished, Available, Finished)


Creating Data Mart: gold_disease_analytics...
📊 KPI - Total Claims with a Valid Disease: 156
✅ Successfully created 'gold_disease_analytics'.


SynapseWidget(Synapse.DataFrame, 49f2b8c9-50c3-44e3-a4b0-201fb7a4ccfc)

DataFrame[claim_id: string, claim_date: date, charge: double, duration: int, insured_id: string, policyholder_id: string, policyholder_full_name: string, policyholder_risk_score: int, high_risk: int, service_id: string, service_name: string, incharge_id: string, incharge_full_name: string, disease_id: string, disease_name: string, is_customer_valid: boolean, is_disease_valid: boolean, claim_year_month: string, claim_year: int, risk_category: string, duration_category: string]

In [3]:
from delta.tables import DeltaTable

gold_tables = [
    "claim_details",
    "policyholder_analytics",
    "service_performance_monthly",
    "overall_kpi",
    "disease_analytics"
]

base_path = "Tables/Gold"

print(f"\n{'='*20} GOLD TABLE SCHEMA VERIFICATION {'='*20}")

for table in gold_tables:
    table_path = f"{base_path}/{table}"
    print(f"\n--- Table Name: {table} ---")
    
    if DeltaTable.isDeltaTable(spark, table_path):
        df = spark.read.format("delta").load(table_path)
        df.printSchema()
    else:
        print(f"⚠️  Table {table} not found at path: {table_path}")


StatementMeta(, c07fd596-9044-4764-820d-b4fba882d8a6, 5, Finished, Available, Finished)


==================== GOLD TABLE SCHEMA VERIFICATION ====================

--- Table Name: claim_details ---
root
 |-- claim_id: string (nullable = true)
 |-- claim_date: date (nullable = true)
 |-- charge: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- insured_id: string (nullable = true)
 |-- policyholder_id: string (nullable = true)
 |-- policyholder_full_name: string (nullable = true)
 |-- policyholder_risk_score: integer (nullable = true)
 |-- high_risk: integer (nullable = true)
 |-- service_id: string (nullable = true)
 |-- service_name: string (nullable = true)
 |-- incharge_id: string (nullable = true)
 |-- incharge_full_name: string (nullable = true)
 |-- disease_id: string (nullable = true)
 |-- disease_name: string (nullable = true)
 |-- is_customer_valid: boolean (nullable = true)
 |-- is_disease_valid: boolean (nullable = true)
 |-- claim_year_month: string (nullable = true)
 |-- claim_year: integer (nullable = true)
 |-- risk_category: string (nul